In [1]:
import h5py
import numpy as np
import pandas as pd
import tensorflow as tf
import sys
import time
import sklearn
from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
#from keras.utils.np_utils import to_categorical
#from sklearn.preprocessing import Normalizer
from keras import callbacks
#from itertools import cycle
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
#from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [2]:
pheno_train = pd.read_csv(r'C:\Users\Soft Zone\Downloads\Compressed\comp_data_2\pheno_train.csv')
pheno_test = pd.read_csv(r'C:\Users\Soft Zone\Downloads\Compressed\comp_data_2\pheno_test.csv')
ss = pd.read_csv(r'C:\Users\Soft Zone\Downloads\Compressed\comp_data_2\sample_submission.csv')
func = h5py.File(r'C:\Users\Soft Zone\Downloads\Compressed\comp_data_2\abide.hdf5', 'r')

# adjust according to submission format
pheno_train['DX_GROUP'] = pheno_train['DX_GROUP'] - 1.0

In [3]:
#pheno_train['DX_GROUP']

In [4]:
pheno_train

,Unnamed: 0,SUB_ID,X,subject,SITE_ID,FILE_ID,DX_GROUP,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,...,qc_notes_rater_1,qc_anat_rater_2,qc_anat_notes_rater_2,qc_func_rater_2,qc_func_notes_rater_2,qc_anat_rater_3,qc_anat_notes_rater_3,qc_func_rater_3,qc_func_notes_rater_3,SUB_IN_SMP
0,814,51131,815,51131,NYU,NYU_0051131,1.0,19.7300,1,NaN,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1
1,162,50237,163,50237,TRINITY,Trinity_0050237,0.0,21.4200,1,R,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1
2,863,51185,864,51185,STANFORD,Stanford_0051185,1.0,8.2464,1,R,...,NaN,maybe,skull-striping fail;,OK,NaN,OK,NaN,OK,NaN,1
3,668,50978,669,50978,NYU,NYU_0050978,0.0,9.5800,1,NaN,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,0
4,508,50653,509,50653,CMU,CMU_a_0050653,0.0,30.0000,1,R,...,ventral edge is cropped,OK,NaN,fail,ic-frontal-temporal-cerebellum,OK,NaN,OK,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926,1068,51482,1069,51482,CALTECH,Caltech_0051482,1.0,21.1000,2,R,...,NaN,OK,NaN,maybe,Ic-parietal-minor,OK,NaN,OK,NaN,0
927,201,50278,202,50278,UM_1,UM_1_0050278,0.0,11.3000,2,R,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,0
928,385,50477,386,50477,USM,USM_0050477,0.0,20.1807,1,NaN,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1
929,655,50965,656,50965,NYU,NYU_0050965,0.0,9.2500,1,NaN,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,1


In [5]:
pheno_test

,Unnamed: 0,SUB_ID,X,subject,SITE_ID,FILE_ID,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,HANDEDNESS_SCORES,...,qc_notes_rater_1,qc_anat_rater_2,qc_anat_notes_rater_2,qc_func_rater_2,qc_func_notes_rater_2,qc_anat_rater_3,qc_anat_notes_rater_3,qc_func_rater_3,qc_func_notes_rater_3,SUB_IN_SMP
0,937,51260,938,51260,UCLA_1,UCLA_1_0051260,13.6600,1,R,NaN,...,NaN,maybe,skull-striping fail;,OK,NaN,OK,NaN,OK,NaN,1
1,123,50184,124,50184,SDSU,SDSU_0050184,17.1500,1,R,NaN,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,0
2,975,51306,976,51306,UCLA_2,UCLA_2_0051306,11.6600,1,R,NaN,...,NaN,maybe,skull-striping fail;,maybe,NaN,OK,NaN,OK,NaN,1
3,634,50818,635,50818,KKI,KKI_0050818,11.7900,1,R,75.0,...,NaN,maybe,no left cerebellum,OK,NaN,OK,NaN,OK,NaN,1
4,1077,51491,1078,51491,CALTECH,Caltech_0051491,56.2000,1,R,NaN,...,NaN,OK,NaN,OK,NaN,OK,NaN,OK,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,321,50408,322,50408,UM_2,UM_2_0050408,14.8000,1,R,NaN,...,NaN,OK,stripe,OK,NaN,OK,NaN,OK,NaN,1
100,151,50212,152,50212,SDSU,SDSU_0050212,13.8100,1,L,NaN,...,NaN,maybe,motion,OK,NaN,fail,noise,OK,NaN,1
101,1078,51492,1079,51492,CALTECH,Caltech_0051492,18.7000,1,Ambi,NaN,...,NaN,OK,NaN,fail,Ic-parietal,OK,NaN,OK,NaN,0
102,422,50514,423,50514,USM,USM_0050514,21.4018,1,NaN,100.0,...,NaN,OK,NaN,maybe,ic-parietal slight,OK,NaN,OK,NaN,1


In [6]:
def get_data_train(data, pheno, derivative):
    X = []
    y = []
    i = 0
    total = pheno.shape[0]
    for row in pheno.iterrows():
        file_id, dx_group = row[1]['FILE_ID'], row[1]['DX_GROUP']
        connectivity = data['patients'][file_id][derivative][()]
        X.append(connectivity)
        y.append(dx_group)
        sys.stdout.write("\r{:.2f}%>".format(i/total))
        sys.stdout.flush()
        i += 1
        
    X = np.array(X).astype(np.float32)
    y = np.array(y).astype(np.float32)
    return X, y


def get_data_test(data, pheno, derivative):
    X_test = []
    sub_ids = []
    j = 0
    total = pheno.shape[0]
    for row in pheno.iterrows():
        file_id, sub_id = row[1]['FILE_ID'], row[1]['SUB_ID']
        connectivity = data['patients'][file_id][derivative][()]
        X_test.append(connectivity)
        sub_ids.append(sub_id)
        sys.stdout.write("\r{:.2f}%>".format(j/total))
        sys.stdout.flush()
        j += 1
        
    X_test = np.array(X_test).astype(np.float32)
    return X_test, sub_ids



In [7]:
X, y = get_data_train(func, pheno_train, 'aal')
X.shape, y.shape

1.00%>

((931, 6670), (931,))

In [8]:
X_test, sub_ids = get_data_test(func, pheno_test, 'aal')
X_test.shape, len(sub_ids)

0.99%>

((104, 6670), 104)

In [9]:
"""%%time
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier as hgbc
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import gc

N_SPLITS = 10
kf = KFold(n_splits=N_SPLITS)
final_preds = np.zeros((X_test.shape[0], N_SPLITS))
i = 0
for tr_idx, val_idx in kf.split(X):
    clf = hgbc(max_depth=6, max_leaf_nodes=25, verbose=0, max_iter=30)
    clf.fit(X[tr_idx], y[tr_idx])
    final_preds[:, i] = clf.predict_proba(X_test)[:, 1]
    fold_preds = clf.predict_proba(X[val_idx])[:, 1]
    print('Fold: {0}, score: {1}'.format(i, roc_auc_score(y[val_idx], fold_preds)))
    del clf
    gc.collect()
    i += 1"""

"%%time\nfrom sklearn.experimental import enable_hist_gradient_boosting\nfrom sklearn.ensemble import HistGradientBoostingClassifier as hgbc\nfrom sklearn.model_selection import KFold\nfrom sklearn.metrics import roc_auc_score\nimport gc\n\nN_SPLITS = 10\nkf = KFold(n_splits=N_SPLITS)\nfinal_preds = np.zeros((X_test.shape[0], N_SPLITS))\ni = 0\nfor tr_idx, val_idx in kf.split(X):\n    clf = hgbc(max_depth=6, max_leaf_nodes=25, verbose=0, max_iter=30)\n    clf.fit(X[tr_idx], y[tr_idx])\n    final_preds[:, i] = clf.predict_proba(X_test)[:, 1]\n    fold_preds = clf.predict_proba(X[val_idx])[:, 1]\n    print('Fold: {0}, score: {1}'.format(i, roc_auc_score(y[val_idx], fold_preds)))\n    del clf\n    gc.collect()\n    i += 1"

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =0.2, random_state = 42)

In [11]:
X_train.shape , y_train.shape

((744, 6670), (744,))

In [12]:
X_test.shape , y_test.shape

((187, 6670), (187,))

In [13]:
"""# Define simple neural network model

from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam,RMSprop
import tensorflow as tf

model_1 = Sequential()
model_1.add(Dense(64, input_dim=X_test.shape[1], activation='relu'))
model_1.add(Dense(16, activation='relu'))
model_1.add(Dropout(0.2))
model_1.add(Dense(1, activation='sigmoid'))


model_1.summary()"""

"# Define simple neural network model\n\nfrom keras.models import Sequential\nfrom keras.layers import Dense\nfrom tensorflow.keras.optimizers import Adam,RMSprop\nimport tensorflow as tf\n\nmodel_1 = Sequential()\nmodel_1.add(Dense(64, input_dim=X_test.shape[1], activation='relu'))\nmodel_1.add(Dense(16, activation='relu'))\nmodel_1.add(Dropout(0.2))\nmodel_1.add(Dense(1, activation='sigmoid'))\n\n\nmodel_1.summary()"

In [14]:
"""callback = tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5',monitor='val_acc', mode='max',verbose=1)

es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)"""

"callback = tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5',monitor='val_acc', mode='max',verbose=1)\n\nes = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)"

In [15]:
"""# Compile the Neural network
model_1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit to training data
history = model_1.fit(X_train, y_train, epochs=100,validation_split=0.2, callbacks=[callback , es])
#history = model_1.fit(X_train, y_train, epochs=100, validation_data=(X_test,y_test), callbacks=[callback , es])"""

"# Compile the Neural network\nmodel_1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])\n# Fit to training data\nhistory = model_1.fit(X_train, y_train, epochs=100,validation_split=0.2, callbacks=[callback , es])\n#history = model_1.fit(X_train, y_train, epochs=100, validation_data=(X_test,y_test), callbacks=[callback , es])"

In [16]:
"""from sklearn.metrics import confusion_matrix, classification_report
predictions = model_1.predict(X_test)
predictions=(predictions>0.5)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))"""

'from sklearn.metrics import confusion_matrix, classification_report\npredictions = model_1.predict(X_test)\npredictions=(predictions>0.5)\nprint(confusion_matrix(y_test,predictions))\nprint(classification_report(y_test,predictions))'

In [17]:
"""model_1.save("fmri_model1.h5")
print("model saved")"""

'model_1.save("fmri_model1.h5")\nprint("model saved")'

In [18]:
from tensorflow.keras.models import Sequential
from keras.layers import LSTM
model_3 = Sequential()
model_3.add(LSTM(64,input_shape=(X_test.shape[1],1), return_sequences=True)) 
model_3.add(LSTM(64,return_sequences=True))
model_3.add(Dropout(0.3)) 
model_3.add(LSTM(32, return_sequences=False))  
model_3.add(Dense(1))
model_3.add(Activation('sigmoid'))
model_3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 6670, 64)          16896     
                                                                 
 lstm_1 (LSTM)               (None, 6670, 64)          33024     
                                                                 
 dropout (Dropout)           (None, 6670, 64)          0         
                                                                 
 lstm_2 (LSTM)               (None, 32)                12416     
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
 activation (Activation)     (None, 1)                 0         
                                                                 
Total params: 62,369
Trainable params: 62,369
Non-traina

In [19]:
callback = tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5',monitor='val_acc', mode='max',verbose=1)

es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

In [20]:
model_3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model_3.fit(X_train, y_train, epochs=100,validation_split=0.2, callbacks=[callback , es])


Epoch 1/100
19/19 [==============================] - ETA: 0s - loss: 0.6946 - accuracy: 0.4958 
Epoch 1: saving model to best_model.h5
19/19 [==============================] - 247s 13s/step - loss: 0.6946 - accuracy: 0.4958 - val_loss: 0.6944 - val_accuracy: 0.4631
Epoch 2/100
19/19 [==============================] - ETA: 0s - loss: 0.6928 - accuracy: 0.5210 
Epoch 2: saving model to best_model.h5
19/19 [==============================] - 272s 14s/step - loss: 0.6928 - accuracy: 0.5210 - val_loss: 0.6984 - val_accuracy: 0.4631
Epoch 3/100
19/19 [==============================] - ETA: 0s - loss: 0.6926 - accuracy: 0.5210 
Epoch 3: saving model to best_model.h5
19/19 [==============================] - 315s 17s/step - loss: 0.6926 - accuracy: 0.5210 - val_loss: 0.6968 - val_accuracy: 0.4631
Epoch 4/100
19/19 [==============================] - ETA: 0s - loss: 0.6939 - accuracy: 0.5042 
Epoch 4: saving model to best_model.h5
19/19 [==============================] - 340s 18s/step - loss: 0.69

In [21]:
from sklearn.metrics import confusion_matrix, classification_report
predictions = model_3.predict(X_test)
predictions=(predictions>0.5)
print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))

6/6 [==============================] - 12s 2s/step
[[ 0 89]
 [ 0 98]]
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00        89
         1.0       0.52      1.00      0.69        98

    accuracy                           0.52       187
   macro avg       0.26      0.50      0.34       187
weighted avg       0.27      0.52      0.36       187



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [23]:
model_3.save("fmri_model3.h5")
print("model saved")

model saved
